# Dengue prediction and Analysis

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/dengue dataset csv.csv


In [2]:
import pandas as pd
import seaborn as sns

## read data

In [3]:
bcdf=pd.read_csv("/kaggle/input/dengue dataset csv.csv")

## top 5 rows

In [ ]:
bcdf.head()

In [ ]:
bcdf.drop(columns={'id'},inplace=True)

In [ ]:
get={'yes':1,'medium':1,'no':0,'high':2,'low':0}


In [ ]:
bcdf.columns

## data cleaning

In [ ]:
bcdf.nausea=bcdf.nausea.map(get) 

In [ ]:
bcdf=bcdf[:200]

In [ ]:
bcdf.nausea

In [ ]:
bcdf.vomiting_blood=bcdf.vomiting_blood.map(get)

In [ ]:
bcdf.vomiting_blood

In [ ]:
bcdf.body_pains=bcdf.body_pains.map(get)

In [ ]:
bcdf.body_pains

In [ ]:
bcdf.pain_behind_eyes=bcdf.pain_behind_eyes.map(get)

In [ ]:
bcdf.joint_pains=bcdf.joint_pains.map(get)

In [ ]:
bcdf.chill=bcdf.chill.map(get)

In [ ]:
bcdf.headache=bcdf.headache.map(get)

In [ ]:
bcdf.rashes=bcdf.rashes.map(get)

In [ ]:
bcdf.ble_nose=bcdf.ble_nose.map(get)

In [ ]:
bcdf.ble_mouth=bcdf.ble_mouth.map(get)

In [ ]:
bcdf.dry_mouth=bcdf.dry_mouth.map(get)

In [ ]:
bcdf.fatigue=bcdf.fatigue.map(get)

In [ ]:
bcdf.red_eyes=bcdf.red_eyes.map(get)

In [ ]:
bcdf.dengue=bcdf.dengue.map(get)

In [ ]:
bcdf.head()

In [ ]:
bcdf.vomiting=bcdf.vomiting.map(get)


In [ ]:
bcdf.swollen_glands=bcdf.swollen_glands.map(get)


In [ ]:
bcdf.abdominal_pain=bcdf.abdominal_pain.map(get)

In [ ]:
bcdf.head()

In [ ]:

bcdf_n = bcdf[bcdf['dengue'] == 0]
bcdf_y = bcdf[bcdf['dengue'] == 1]

c=0
for i in bcdf.dengue:
    if i==1:
        c=c+1
score=c/bcdf.shape[0]
print('the bechmark model accuarcy score  {}%'.format(score*100))

## data visualization

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

# Instantiate figure object
fig = plt.figure()
#plt.sup_title('Tumor Characteristic (means)')

# Create 'for loop' to enerate though tumor features and compare with histograms
for i,b in enumerate(list(bcdf.columns[0:9])):
    
    # Enumerate starts at index 0, need to add 1 for subplotting
    i +=1
    
    # Create axes object for position i
    ax = fig.add_subplot(3,3,i)
    
    # Plot via histogram tumor charateristics using stacked and alpha parameters for..
    # comparisons.
    ax.hist(bcdf_n[b], label = 'Negative', stacked = True, alpha=0.5, color= 'g')
    ax.hist(bcdf_y[b], label= 'Positive', stacked = True, alpha=0.5, color= 'r')
    ax.set_title(b)



plt.tight_layout()
plt.legend()
plt.show()   

## heat map generation

In [ ]:
sns.heatmap(bcdf.corr())

## Data Modeling

In [ ]:
## to split the data into ratio of 75% and 25% to train model and test the model

from sklearn.model_selection import train_test_split

new_data=bcdf.drop(columns={'dengue'})
X_train,X_test,y_train,y_test=train_test_split(new_data,bcdf['dengue'],random_state=10)

In [ ]:
#The data is preprocessing using sklearn.preprocessing.Normalizer
from sklearn.preprocessing import Normalizer

norm=Normalizer()
X_train_normal=norm.transform(X_train)
X_test_normal=norm.transform(X_test)

### Hyperparameter tuning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


RF_params = {'n_estimators':[10,50,100]}
DTC_params = {'criterion':['entropy'], 'max_depth':[10, 50, 100]}
LR_params = {'C':[0.001, 0.1, 1, 10, 100]}

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
from sklearn.model_selection import GridSearchCV
models=[]

models.append(('DTC', DecisionTreeClassifier(), DTC_params))

models.append(('LR', LogisticRegression(), LR_params))

In [ ]:
from tqdm import tqdm
results=[]
names=[]
scoring='accuracy' 
for name, model, params in tqdm(models):
    kfold = KFold(len(X_train_normal), random_state=2, shuffle=True)
    model_grid = GridSearchCV(model, params)
    cv_results = cross_val_score(model_grid, X_train_normal, y_train, cv = kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "Cross Validation Accuracy %s: Accarcy: %f SD: %f" % (name, cv_results.mean(), cv_results.std())
    print(msg)

### White Grid Plot

In [ ]:
plt.boxplot(results, labels = names)
plt.title('Dengue Diagnosis Performance using Machine Learning ')
plt.ylabel('Model Accuracy %')
sns.set_style("whitegrid")
plt.show()

### Accuracy

In [ ]:
#The accuracy score obtained without using GridSearchCV

from sklearn.metrics import make_scorer, accuracy_score, fbeta_score

clf=RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_normal,y_train)
pred=clf.predict(X_test_normal)
accuracy_score(pred,y_test)


### Data metric evaluation

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score

clf = DecisionTreeClassifier(random_state=42)

# TODO: Create the parameters list you wish to tune
parameters = {'criterion':['entropy'], 'max_depth':[10, 50, 100]}

# TODO: Make an fbeta_score scoring object


# TODO: Perform grid search on the classifier using 'scorer' as the scoring method
grid_obj = GridSearchCV(clf,parameters,scoring='accuracy')

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_fit = grid_obj.fit(X_train_normal,y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train_normal, y_train)).predict(X_test_normal)
best_predictions = best_clf.predict(X_test_normal)

# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))

###  Confusion Matrix and Classification report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

matrix=confusion_matrix(pred,y_test)
matrix=pd.DataFrame(matrix,columns=['Predicted Negative','Predicted Postive'],index=['Actual Negative','Actual Positive'])
print(matrix)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(best_predictions,y_test))